In [12]:
from scipy import io as sio
import pickle
import gzip
mat = sio.loadmat('emnist-letters.mat')
data = mat['dataset']

X_train = data['train'][0,0]['images'][0,0]
y_train = data['train'][0,0]['labels'][0,0]
X_test = data['test'][0,0]['images'][0,0]
y_train = data['test'][0,0]['labels'][0,0]
val_start = X_train.shape[0] - X_test.shape[0]
X_val = X_train[val_start:X_train.shape[0],:]
y_val = y_train[val_start:X_train.shape[0]]
X_train = X_train[0:val_start,:]
y_train = y_train[0:val_start]

X_train = list(zip(X_train,y_train))

f = gzip.open('emnist.pkl.gz','wb')
pickle.dump(X_train,f)
f.close()

f = gzip.open('emnist.pkl.gz','rb')
myNewObject = pickle.load(f)
f.close()



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

